In [1]:
#score 0.63 - 0.65, good luck for you. Upload if you like it 
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import GRU, Bidirectional, GlobalAveragePooling1D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
import polars as pl
import kaggle_evaluation.cmi_inference_server
print("Imports loaded")

# Load the dataset
print("Loading dataset...")
df = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv')
print(f"Loaded {len(df)} rows.")

label_encoder = LabelEncoder()
df['gesture'] = label_encoder.fit_transform(df['gesture'].astype(str))

# Save class names for inference
np.save('gesture_classes.npy', label_encoder.classes_)

# Print class label mapping
print("Gesture label mapping:")
for idx, label in enumerate(label_encoder.classes_):
    print(f"  {idx}: {label}")

print("Checking for IMU-only sequences...")

def check_for_imu_only_seqs():
    # Identify thermopile and TOF columns
    thermal_tof_cols = [col for col in df.columns if col.startswith('thm_') or col.startswith('tof_')]
    
    # Group by sequence and check if all thm_/tof_ values are null
    imu_only_flags = df[thermal_tof_cols].isna().groupby(df['sequence_id']).all().all(axis=1)
    
    # Report statistics
    total_sequences = df['sequence_id'].nunique()
    imu_only_count = imu_only_flags.sum()
    imu_only_pct = (imu_only_count / total_sequences) * 100
    
    print(f"Total sequences: {total_sequences}")
    print(f"IMU-only sequences (all thm_/tof_ null): {imu_only_count} ({imu_only_pct:.1f}%)")

check_for_imu_only_seqs()

excluded_cols = {
    'gesture', 'sequence_type', 'behavior', 'orientation',  # train-only
    'row_id', 'subject', 'phase',  # metadata
    'sequence_id', 'sequence_counter'  # identifiers
}

# Setting this true makes model ignore thermal and tof data
drop_thermal_and_tof = True

if drop_thermal_and_tof:
    thermal_tof_cols = [col for col in df.columns if col.startswith('thm_') or col.startswith('tof_')]
    excluded_cols.update(thermal_tof_cols)
    print(f"Ignoring {len(thermal_tof_cols)} thermopile / time-of-flight columns.")

# Select numeric feature columns
feature_cols = [col for col in df.columns if col not in excluded_cols]
print(f"Using {len(feature_cols)} numeric feature columns for training:")
print(feature_cols)

# Check for NaNs in selected feature columns
nan_counts = df[feature_cols].isna().sum()
total_nans = nan_counts.sum()
print(f"\nTotal missing values in feature columns: {total_nans}")
if total_nans > 0:
    print("Columns with missing values:")
    print(nan_counts[nan_counts > 0])
else:
    print("No missing values found in feature columns.")

def preprocess_sequence(df_sequence: pd.DataFrame, feature_cols: list) -> np.ndarray:
    data = df_sequence[feature_cols].copy()
    data = data.ffill().bfill().fillna(0)
    scaled = StandardScaler().fit_transform(data)
    return scaled

# Build sequences
sequence_ids = df['sequence_id'].unique()
sequences = df.groupby('sequence_id')

X = []
seq_lengths = []

print("Building sequences...")
for i, (seq_id, seq) in enumerate(sequences):
    if i % 500 == 0:
        print(f"Processing sequence {i}...")
    processed = preprocess_sequence(seq, feature_cols)
    X.append(processed)
    seq_lengths.append(processed.shape[0])

max_len_perentile = 90

# Report sequence length stats
minlen = min(seq_lengths)
avglen = int(np.mean(seq_lengths))
pad_len_to_use = int(np.percentile(seq_lengths, max_len_perentile))  
print(f"Sequence length stats - Min: {minlen}, Avg: {avglen}, {max_len_perentile}th percentile: {pad_len_to_use}")
print(f"Padding / truncating all sequences to fixed length {pad_len_to_use}...")

np.save("sequence_maxlen.npy", pad_len_to_use)  # Save for inference

# Pad/truncate to fixed length
X = pad_sequences(X, maxlen=pad_len_to_use, dtype='float32', padding='post', truncating='post')

# Use groupby to get the first gesture per sequence (already integer-encoded)
y = df.groupby('sequence_id')['gesture'].first().values

print("Integer labels:", y[:4])

# Convert to one-hot vectors
num_classes = len(np.unique(y))
y = to_categorical(y, num_classes=num_classes)

print("After one-hot encoding:", y[:4])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Build CNN-GRU model
print("Building CNN-GRU model...")
model = Sequential([
    # Multi-scale CNN blocks for feature extraction
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    BatchNormalization(),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(2),
    Dropout(0.3),
    
    Conv1D(128, kernel_size=5, activation='relu'),
    BatchNormalization(),
    Conv1D(128, kernel_size=5, activation='relu'),
    MaxPooling1D(2),
    Dropout(0.3),
    
    Conv1D(256, kernel_size=7, activation='relu'),
    BatchNormalization(),
    Conv1D(256, kernel_size=7, activation='relu'),
    MaxPooling1D(2),
    Dropout(0.4),
    
    # Bidirectional GRU for temporal dependencies (more efficient than LSTM)
    Bidirectional(GRU(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),
    Bidirectional(GRU(64, return_sequences=False, dropout=0.3, recurrent_dropout=0.3)),
    
    # Dense layers for classification
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(128, activation='relu'),
    Dropout(0.3),
    
    Dense(num_classes, activation='softmax')
])

# Compile model with categorical crossentropy loss (for one-hot labels)
model.compile(
    optimizer=AdamW(learning_rate=0.001, weight_decay=1e-4), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)
model.summary()

# Define callbacks for better training
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
]

# Train model using explicitly split validation set (80/20 held out)
print("Training CNN-GRU model...")
history = model.fit(
    X_train, y_train, 
    epochs=60, 
    batch_size=128, 
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)
model.save("gesture_cnn_gru_model.h5")
print("Training complete.")

from cmi_2025_metric_copy_for_import import CompetitionMetric

# Get predicted labels for the validation set
print("Predicting on validation set...")
y_val_pred_probs = model.predict(X_val, verbose=0)
y_val_pred = np.argmax(y_val_pred_probs, axis=1)
y_val_true = np.argmax(y_val, axis=1)

# Map integer labels back to gesture strings
gesture_classes = np.load("gesture_classes.npy", allow_pickle=True)
val_pred_labels = pd.Series(y_val_pred).map(lambda i: gesture_classes[i])
val_true_labels = pd.Series(y_val_true).map(lambda i: gesture_classes[i])

# Build DataFrames for the metric
val_submission = pd.DataFrame({'gesture': val_pred_labels})
val_solution = pd.DataFrame({'gesture': val_true_labels})

# Run competition metric
metric = CompetitionMetric()
score = metric.calculate_hierarchical_f1(val_solution, val_submission)
print(f"Estimated leaderboard (val) score: {score:.4f}")

def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    df_seq = sequence.to_pandas()
    processed = preprocess_sequence(df_seq, feature_cols)
    maxlen = int(np.load("sequence_maxlen.npy"))  # ensure consistent shape
    padded = pad_sequences([processed], maxlen=maxlen, dtype='float32', padding='post', truncating='post')
    model = load_model("gesture_cnn_gru_model.h5")
    prediction = model.predict(padded, verbose=0)
    predicted_index = np.argmax(prediction, axis=1)[0]
    gesture_classes = np.load("gesture_classes.npy", allow_pickle=True)
    return gesture_classes[predicted_index]

# Launch inference server
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        data_paths=(
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
        )
    )

# Manual test (only runs outside Kaggle gateway)
if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("\nRunning manual test...")
    test_df = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv')
    sample_seq_id = test_df['sequence_id'].unique()[0]
    test_seq = test_df[test_df['sequence_id'] == sample_seq_id]
    prediction = predict(pl.DataFrame(test_seq), None)
    print(f"Manual prediction result for sequence_id {sample_seq_id}: {prediction}")

2025-06-01 12:21:40.162695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748780500.342929      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748780500.395785      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Imports loaded
Loading dataset...
Loaded 574945 rows.
Gesture label mapping:
  0: Above ear - pull hair
  1: Cheek - pinch skin
  2: Drink from bottle/cup
  3: Eyebrow - pull hair
  4: Eyelash - pull hair
  5: Feel around in tray and pull out an object
  6: Forehead - pull hairline
  7: Forehead - scratch
  8: Glasses on/off
  9: Neck - pinch skin
  10: Neck - scratch
  11: Pinch knee/leg skin
  12: Pull air toward your face
  13: Scratch knee/leg skin
  14: Text on phone
  15: Wave hello
  16: Write name in air
  17: Write name on leg
Checking for IMU-only sequences...
Total sequences: 8151
IMU-only sequences (all thm_/tof_ null): 96 (1.2%)
Ignoring 325 thermopile / time-of-flight columns.
Using 7 numeric feature columns for training:
['acc_x', 'acc_y', 'acc_z', 'rot_w', 'rot_x', 'rot_y', 'rot_z']

Total missing values in feature columns: 14768
Columns with missing values:
rot_w    3692
rot_x    3692
rot_y    3692
rot_z    3692
dtype: int64
Building sequences...
Processing sequence 0.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1748780564.320785      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 101, 64)             │           1,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 101, 64)             │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 99, 64)              │          12,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 49, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 49, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 45, 128)             │          41,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 45, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 41, 128)             │          82,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 20, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 20, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 14, 256)             │         229,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 14, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 8, 256)              │         459,008 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 4, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 4, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 4, 256)              │         296,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │         123,648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

...

 Trainable params: 1,480,658 (5.65 MB)

 Non-trainable params: 2,432 (9.50 KB)

Training CNN-GRU model...
Epoch 1/60


I0000 00:00:1748780588.311936      58 cuda_dnn.cc:529] Loaded cuDNN version 90300


51/51 ━━━━━━━━━━━━━━━━━━━━ 32s 116ms/step - accuracy: 0.0659 - loss: 3.6902 - val_accuracy: 0.0681 - val_loss: 2.8114 - learning_rate: 0.0010
Epoch 2/60
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.1079 - loss: 3.0649 - val_accuracy: 0.0797 - val_loss: 2.8555 - learning_rate: 0.0010
Epoch 3/60
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - accuracy: 0.1282 - loss: 2.8594 - val_accuracy: 0.0889 - val_loss: 2.8364 - learning_rate: 0.0010
Epoch 4/60
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.1714 - loss: 2.5980 - val_accuracy: 0.1042 - val_loss: 2.8906 - learning_rate: 0.0010
Epoch 5/60
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.1715 - loss: 2.5089 - val_accuracy: 0.1294 - val_loss: 2.9089 - learning_rate: 0.0010
Epoch 6/60
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.1980 - loss: 2.3842 - val_accuracy: 0.1392 - val_loss: 2.6890 - learning_rate: 0.0010
Epoch 7/60
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.2127 - loss: 2.3022 - val_accuracy: 0.